<a href="https://colab.research.google.com/github/srkhvx/Intelligent-Cyber-Security/blob/master/Copy_of_Section_1_Differential_Privacy_Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db.shape

torch.Size([5000])

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, create a list of every parallel database to the one currently contained in the "db" variable. Then, create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [2]:
db = torch.rand(num_entries) > 0.5
db[:7]

tensor([1, 0, 1, 0, 0, 0, 1], dtype=torch.uint8)

In [0]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [4]:
x = get_parallel_db(db, 1)
print(f'size of x: {x.shape}')
print(x[:5]) # notice the change in from teh original db

size of x: torch.Size([4999])
tensor([1, 1, 0, 0, 0], dtype=torch.uint8)


In [5]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs
  
  
paralled_dbs = get_parallel_dbs(db)

print('number of parallel dbs: ', len(paralled_dbs))
print('size of each parallel db: ', len(paralled_dbs[0]))

# shape is list of lists [[], [], [], ...]

number of parallel dbs:  5000
size of each parallel db:  4999


In [0]:
pdbs = get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(20)

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [0]:
db, pdbs = create_db_and_parallels(5000)

In [48]:
# a sample sum query
def query(db):
    return db.sum()
print(query(db))

tensor(2497)


In [11]:
full_db_result = query(db)
full_db_result

tensor(2456)

In [47]:
# TO DO:
def sensity(db,pdb):
  alldiff=[]
  main=mean_query(db)
  for i in range(len(db)-1):
    anyother=mean_query(pdb[i])
    sensitivity= main - anyother
    alldiff.append(sensitivity)
  return (max(alldiff))
print(sensity(db,pdbs))


tensor(0.0001)


# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [0]:
# To Do: 
# define a function to caluculate the mean of a vector (a pdb). There is a predefined function in torch that does that
def mean_query(data):
  s=query(data.float())
  return (s/len(data))
  

In [14]:
print(mean_query(db))

tensor(0.4912)


In [0]:
def threshold_query(db, threshold=5):
  main=query(db)
  if(main>threshold):
    return 1
  else:
    return 0

In [0]:
def sensitivity(queryx, n_entries,threshold=5):
  db, pdbs = create_db_and_parallels(n_entries)
  if(queryx=="sum"):
    alldiff=[]
    main=query(db)
    for i in range(len(db)-1):
      anyother=query(pdbs[i])
      sensitivityx= main - anyother
      alldiff.append(sensitivityx)
    return(max(alldiff))
  elif(queryx=="mean"):
    alldiff=[]
    main=mean_query(db)
    for i in range(len(db)-1):
      anyother=mean_query(pdbs[i])
      sensitivityx= main - anyother
      alldiff.append(sensitivityx)
    return(max(alldiff))
  elif(queryx=="threshold"):
    alldiff=[]
    main=threshold_query(db,threshold)
    for i in range(len(db)-1):
      anyother=threshold_query(pdbs[i],threshold)
      sensitivityx= main - anyother
      alldiff.append(sensitivityx)
    return(max(alldiff))

In [17]:
# define a fucntion called sensitivity that accepts a query and a number of entries, 
# which then creates a db with number of entires and pdbs and caculates the sensitivity of the query
print(sensitivity("mean",5000))

tensor(9.9242e-05)


Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

Calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database and return whether that sum is greater than a certain threshold.
- Then, create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [0]:
# TO DO
def threshold_query(db, threshold=5):
  main=query(db)
  if(main>threshold):
    return 1
  else:
    return 0

In [19]:
# TO DO
db, pdbs = create_db_and_parallels(10)
print(sensitivity("threshold",10,5))

# find the sensitivity of a db of size 10 and threshold 5

1


In [20]:
# TO DO
for i in range(10):
  db, pdbs = create_db_and_parallels(10)
  print(sensitivity("threshold",10,5))
# re-initialize the database 10 times and calculate the sensitivity each time.

1
0
0
0
0
0
0
0
0
0


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [21]:
# TO DO
# create a db with 100 entries (binaries for simplicity)
db, pdbs = create_db_and_parallels(100)
# check the value at row 10 (in real senario you do not know this in advance)
print("Value of the 10th Person: "+str(db[9]))
# create a differencing attack using the sum query to learn the value at row 10
person=query(db)-query(get_parallel_db(db, 9))
print("Value of the Person: "+str(person))

Value of the 10th Person: tensor(1, dtype=torch.uint8)
Value of the Person: tensor(1)


In [22]:
# create a differencing attack on the mean query to learn the value at row 10
db, pdbs = create_db_and_parallels(100)

print("Value of the 10th Person: "+str(db[9]))
pdbs=get_parallel_db(db, 9)
person=(mean_query(db)-mean_query(pdbs))
if(person<0):
  print("Value of the Person: 0")
else:
    print("Value of the Person: 1")

Value of the 10th Person: tensor(1, dtype=torch.uint8)
Value of the Person: 1


In [23]:
# create a differencing attack on the threshold query to learn the value at row 10
db, pdbs = create_db_and_parallels(10)

print("Value of the 5th Person: "+str(db[5]))
pdbs=get_parallel_db(db, 9)
person=(threshold_query(db,5)-threshold_query(pdbs,5))
if(person<1):
  print("Value of the Person: 0")
else:
    print("Value of the Person: 1")

Value of the 5th Person: tensor(1, dtype=torch.uint8)
Value of the Person: 0


# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [0]:
# actual values from people
import torch
def dataset(length=10,rd=0.5):
  db = torch.rand(length) > rd
  return db

In [0]:
# create a function to implement our Coin flip Randomized Response: 
def randomized_response(db,flips,lastflip,rd):
  noise=(db*flips)+((1-flips)*(lastflip))
  noise_query=mean_query(noise)
  db_query=mean_query(db)
  
  Augmented=(2*(noise_query)-1+rd)/(2*rd)
  return db_query,noise_query,noise,Augmented
  
    
  # returns actual true results (from query (db)), 
  # and the results (query(db_noisy)): this means de-skewing  the augmented result

In [26]:
# print both
db=dataset(10,0.5)
print(db)
flips=dataset(10,0.5)
lastflip=dataset(10,0.5)
db_query,noise_query,noise,Augmented=randomized_response(db,flips,lastflip,0.5)
print("True Result: ",db_query)
print("Noise Result: ",noise_query)
print("Noise: ",noise)

tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1], dtype=torch.uint8)
True Result:  tensor(0.7000)
Noise Result:  tensor(0.3000)
Noise:  tensor([0, 0, 0, 1, 0, 1, 0, 1, 0, 0], dtype=torch.uint8)


In [0]:
# create a new db with 100, 1000, 10000 entries and then repeat the previous steps:
def make_set(lgt=10,rd=0.5):
  db=dataset(lgt,rd)
  flips=dataset(lgt,rd)
  lastflip=dataset(lgt,rd)
  return db, flips, lastflip
# compare the mean query between db and noisy_db of each size
# What's your observation?

In [28]:
rd=0.5
db,flips,lastflip=make_set(100,rd);
db_query,noise_query,noise,Aug=randomized_response(db,flips,lastflip,rd)
print("True Result for 100: ",db_query)
print("Noise Result for 100: ",noise_query)
print("Augmneted Result for 100: ",Aug)
print("Epsilon for 100: ",(noise_query-Aug))


db,flips,lastflip=make_set(1000,0.5);
db_query,noise_query,noise,Aug=randomized_response(db,flips,lastflip,rd)
print("True Result for 1000: ",db_query)
print("Noise Result for 1000: ",noise_query)
print("Augmneted Result for 1000: ",Aug)
print("Epsilon for 1000: ",(noise_query-Aug))


db,flips,lastflip=make_set(10000,0.5);
db_query,noise_query,noise,Aug=randomized_response(db,flips,lastflip,rd)
print("True Result for 10000: ",db_query)
print("Noise Result for 10000: ",noise_query)
print("Augmneted Result for 10000: ",Aug)
print("Epsilon for 10000: ",(noise_query-Aug))


db,flips,lastflip=make_set(100000,0.5);
db_query,noise_query,noise,Aug=randomized_response(db,flips,lastflip,rd)
print("True Result for 100000: ",db_query)
print("Noise Result for 100000: ",noise_query)
print("Augmneted Result for 100000: ",Aug)
print("Epsilon for 10000: ",(noise_query-Aug))


True Result for 100:  tensor(0.5600)
Noise Result for 100:  tensor(0.4400)
Augmneted Result for 100:  tensor(0.3800)
Epsilon for 100:  tensor(0.0600)
True Result for 1000:  tensor(0.5180)
Noise Result for 1000:  tensor(0.5140)
Augmneted Result for 1000:  tensor(0.5280)
Epsilon for 1000:  tensor(-0.0140)
True Result for 10000:  tensor(0.4951)
Noise Result for 10000:  tensor(0.4975)
Augmneted Result for 10000:  tensor(0.4950)
Epsilon for 10000:  tensor(0.0025)
True Result for 100000:  tensor(0.4981)
Noise Result for 100000:  tensor(0.4971)
Augmneted Result for 100000:  tensor(0.4942)
Epsilon for 10000:  tensor(0.0029)


# Project: Varying Amounts of Noise and Apporximation

In this project, you need to augment the randomized response query, i.e., `randomized_response`,  to allow for varying amounts of randomness to be added. Specifically, to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND approximating the result at the end (where we create the "augmented_result" variable).

In [0]:
# create a function to implement our Coin flip Randomized Response: 
db50,flips50,lastflip50=make_set(100,(1-0.5));
db20,flips20,lastflip20=make_set(100,(1-0.2));
db75,flips75,lastflip75=make_set(100,(1-0.75));
  # returns true result and augmented result

In [30]:
# try the same query with diff amounts of noise [0.2, 0.5, 0.75] and dataset [100, 1000, 10000]
rd=0.2
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips20,lastflip50,rd)
print("True Result for 100 for 20% ",db_query)
print("Noise Result for 100 for 20% ",noise_query)
print("Augmneted Result for 100 20% ",Aug)
print("Epsilon for 100 for 20% ",(noise_query-Aug))

rd=0.5
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips50,lastflip50,rd)
print("True Result for 100 for 50% ",db_query)
print("Noise Result for 100 for 50% ",noise_query)
print("Augmneted Result for 100 50% ",Aug)
print("Epsilon for 100 for 50% ",(noise_query-Aug))

rd=0.75
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips75,lastflip50,rd)
print("True Result for 100 for 75% ",db_query)
print("Noise Result for 100 for 75% ",noise_query)
print("Augmneted Result for 100 75% ",Aug)
print("Epsilon for 100 for 75% ",(noise_query-Aug))


True Result for 100 for 20%  tensor(0.4100)
Noise Result for 100 for 20%  tensor(0.5000)
Augmneted Result for 100 20%  tensor(0.5000)
Epsilon for 100 for 20%  tensor(0.)
True Result for 100 for 50%  tensor(0.4100)
Noise Result for 100 for 50%  tensor(0.3800)
Augmneted Result for 100 50%  tensor(0.2600)
Epsilon for 100 for 50%  tensor(0.1200)
True Result for 100 for 75%  tensor(0.4100)
Noise Result for 100 for 75%  tensor(0.4000)
Augmneted Result for 100 75%  tensor(0.1000)
Epsilon for 100 for 75%  tensor(0.3000)


In [0]:
# create a function to implement our Coin flip Randomized Response: 
db50,flips50,lastflip50=make_set(1000,(1-0.5));
db20,flips20,lastflip20=make_set(1000,(1-0.2));
db75,flips75,lastflip75=make_set(1000,(1-0.75));
  # returns true result and augmented result

In [32]:
# try the same query with diff amounts of noise [0.2, 0.5, 0.75] and dataset [100, 1000, 10000]
rd=0.2
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips20,lastflip50,rd)
print("True Result for 1000 for 20% ",db_query)
print("Noise Result for 1000 for 20% ",noise_query)
print("Augmneted Result for 1000 20% ",Aug)
print("Epsilon for 1000 for 20% ",(noise_query-Aug))

rd=0.5
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips50,lastflip50,rd)
print("True Result for 1000 for 50% ",db_query)
print("Noise Result for 1000 for 50% ",noise_query)
print("Augmneted Result for 1000 50% ",Aug)
print("Epsilon for 1000 for 50% ",(noise_query-Aug))

rd=0.75
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips75,lastflip50,rd)
print("True Result for 1000 for 75% ",db_query)
print("Noise Result for 1000 for 75% ",noise_query)
print("Augmneted Result for 1000 75% ",Aug)
print("Epsilon for 1000 for 75% ",(noise_query-Aug))

True Result for 1000 for 20%  tensor(0.4900)
Noise Result for 1000 for 20%  tensor(0.4750)
Augmneted Result for 1000 20%  tensor(0.4688)
Epsilon for 1000 for 20%  tensor(0.0062)
True Result for 1000 for 50%  tensor(0.4900)
Noise Result for 1000 for 50%  tensor(0.4970)
Augmneted Result for 1000 50%  tensor(0.4940)
Epsilon for 1000 for 50%  tensor(0.0030)
True Result for 1000 for 75%  tensor(0.4900)
Noise Result for 1000 for 75%  tensor(0.5000)
Augmneted Result for 1000 75%  tensor(0.5000)
Epsilon for 1000 for 75%  tensor(0.)


In [0]:
db50,flips50,lastflip50=make_set(10000,(1-0.5));
db20,flips20,lastflip20=make_set(10000,(1-0.2));
db75,flips75,lastflip75=make_set(10000,(1-0.75));
  # returns true result and augmented result

In [34]:
rd=0.2
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips20,lastflip50,rd)
print("True Result for 10000 for 20% ",db_query)
print("Noise Result for 10000 for 20% ",noise_query)
print("Augmneted Result for 10000 20% ",Aug)
print("Epsilon for 10000 for 20% ",(noise_query-Aug))

rd=0.5
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips50,lastflip50,rd)
print("True Result for 10000 for 50% ",db_query)
print("Noise Result for 10000 for 50% ",noise_query)
print("Augmneted Result for 10000 50% ",Aug)
print("Epsilon for 10000 for 50% ",(noise_query-Aug))

rd=0.75
rd=1-rd
db_query,noise_query,noise,Aug=randomized_response(db50,flips75,lastflip50,rd)
print("True Result for 10000 for 75% ",db_query)
print("Noise Result for 10000 for 75% ",noise_query)
print("Augmneted Result for 10000 75% ",Aug)
print("Epsilon for 10000 for 75% ",(noise_query-Aug))

True Result for 10000 for 20%  tensor(0.5030)
Noise Result for 10000 for 20%  tensor(0.5065)
Augmneted Result for 10000 20%  tensor(0.5081)
Epsilon for 10000 for 20%  tensor(-0.0016)
True Result for 10000 for 50%  tensor(0.5030)
Noise Result for 10000 for 50%  tensor(0.5055)
Augmneted Result for 10000 50%  tensor(0.5110)
Epsilon for 10000 for 50%  tensor(-0.0055)
True Result for 10000 for 75%  tensor(0.5030)
Noise Result for 10000 for 75%  tensor(0.4999)
Augmneted Result for 10000 75%  tensor(0.4996)
Epsilon for 10000 for 75%  tensor(0.0003)


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [0]:
import numpy as np
import torch

In [0]:
def sum_query(db):
    return db.sum()

In [0]:
db, pdb = create_db_and_parallels(100)

In [0]:
# create an augmented mechanism that adds Global noise to a given query using laplace type
def noise_mechanism(db, query, sensitivity, epsilon=0.5):
  # laplasian noise depends on a beta variable as follows
  b = sensitivity / epsilon

  # noise amout of noise to be added
  noise = torch.tensor(np.random.laplace(0, scale=b, size=1))

  noisy_result = query(db.float()) + noise.float()

  return noisy_result

In [54]:
# true result which anlayst does not have access to:
print(f'True query result: {sum_query(db).item()}')

True query result: 52


In [51]:
sense=sensity(db,pdb)
print(sense)

tensor(0.0048)


In [82]:
result = noise_mechanism(db, sum_query, sense)
print(f'Augmented, or noisy, result for Sum Query: {result.item()}')

AttributeError: ignored

In [67]:
# TODO
result = noise_mechanism(db, mean_query, sense)
print(f'True Mean query result:                     {mean_query(db).item()}')
print(f'Augmented, or noisy, result for Mean Query: {result.item()}')
# Repeat the previous experiment for the Mean query

True Mean query result:                     0.5199999809265137
Augmented, or noisy, result for Mean Query: 0.5151166915893555


In [69]:
# TODO
# decrease the value of epsilon to 0.01 and then to 0.0001

# TODO
result = noise_mechanism(db, mean_query, sense,0.5)
print(f'True Mean query result:                     {mean_query(db).item()}')
print(f'Augmented, or noisy, result for Mean Query With Epsilon 0.5: {result.item()}')

result = noise_mechanism(db, mean_query, sense,0.01)
print(f'True Mean query result:                     {mean_query(db).item()}')
print(f'Augmented, or noisy, result for Mean Query With Epsilon 0.01: {result.item()}')

result = noise_mechanism(db, mean_query, sense,0.0001)
print(f'True Mean query result:                     {mean_query(db).item()}')
print(f'Augmented, or noisy, result for Mean Query With Epsilon 0.0001: {result.item()}')

# What happesn to the amount of noise?

True Mean query result:                     0.5199999809265137
Augmented, or noisy, result for Mean Query With Epsilon 0.5: 0.5211960077285767
True Mean query result:                     0.5199999809265137
Augmented, or noisy, result for Mean Query With Epsilon 0.01: 0.8918602466583252
True Mean query result:                     0.5199999809265137
Augmented, or noisy, result for Mean Query With Epsilon 0.0001: -5.243734359741211


# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [0]:
import numpy as np

In [0]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [0]:
def create_db_and_parallels_10(num_entries):
    
    db = torch.rand(num_entries)>
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [109]:
db=np.random.randint(10, size=(10000, 10))
newlabels=[]
oldlabels=[]
diff=[]
for i in range(len(db)):
  #print("Before Removing Label: ",db[i])
  oldlabels.append(np.argmax(np.bincount(db[i], minlength=10)))
  #print("Most repeated Label: ",oldlabels[i])
  x=db[i]
  x=np.delete(x,np.random.randint(10))
  #print("After Removing one random Label: ", x)
  newlabels.append(np.argmax(np.bincount(x, minlength=9)))
  #print("Most repeated Label: ",newlabels[i])
  diff.append(abs(newlabels[i]-oldlabels[i]))
  
sensitive=max(diff)
print("MAX DIFFERENCE AKA SENSITIVITY: ",sensitive)
print("Calculating Noise witht this above Sensitivity...")
ep=1
print("For Epsilon: ",ep)
b= sensitive/ep
n = torch.tensor(np.random.laplace(0, scale=b, size=1))
print(n)
#print(newlabels)
#newlabels=oldlabels+n[0].item()
#print(newlabels)
for i in range(len(oldlabels)):
  newlabels[i]=abs(oldlabels[i]+n[0].item())
print(newlabels)

MAX DIFFERENCE AKA SENSITIVITY:  9
Calculating Noise witht this above Sensitivity...
For Epsilon:  1
tensor([-11.1925], dtype=torch.float64)
[11.192459051452449, 10.192459051452449, 8.192459051452449, 7.192459051452449, 4.192459051452449, 11.192459051452449, 3.192459051452449, 7.192459051452449, 7.192459051452449, 7.192459051452449, 4.192459051452449, 3.192459051452449, 7.192459051452449, 10.192459051452449, 10.192459051452449, 8.192459051452449, 8.192459051452449, 11.192459051452449, 6.192459051452449, 10.192459051452449, 5.192459051452449, 10.192459051452449, 8.192459051452449, 9.192459051452449, 7.192459051452449, 6.192459051452449, 9.192459051452449, 8.192459051452449, 11.192459051452449, 7.192459051452449, 3.192459051452449, 11.192459051452449, 6.192459051452449, 8.192459051452449, 8.192459051452449, 11.192459051452449, 4.192459051452449, 8.192459051452449, 4.192459051452449, 11.192459051452449, 8.192459051452449, 11.192459051452449, 8.192459051452449, 2.192459051452449, 11.192459

# PATE Analysis

In [0]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [0]:
from syft.frameworks.torch.differential_privacy import pate

In [0]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps



In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [0]:
preds[:,0:50] *= 0

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to be given a dataset which you need to use to train a DP model using this PATE method. 